#Linear Regression

In [15]:
import numpy as np
import torch

In [16]:
#Input and output
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [17]:
# Conversion to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


##LR from scratch using Gradient Descent Algo

In [18]:
#Intailizing weigth and biases randomly
w = torch.randn(2,3,requires_grad=True) #two outputs and 3 inputs
b = torch.randn(2,requires_grad=True)#Two outputs

In [19]:
#Define the model
def model(x):
  return x @ w.t() + b
# @ means matrix multiplication and w.t() means transpose of w

In [20]:
#Gereating the predictions
preds = model(inputs)

In [21]:
#Define the loss
def mse(preds,actual):
  diff = preds-actual
  return torch.sum(diff*diff) / diff.numel()
# diff.numel to get the number of elements in the vector

In [22]:
loss = mse(preds,targets)

In [23]:
#Computing the gradient of the loss function with respect to the weights
loss.backward()
print(w.grad)

tensor([[  1487.1799,    980.6584,    676.3719],
        [-11213.4707, -12471.2373,  -7883.8643]])


In [24]:
#Reducing the loss
with torch.no_grad():
  w-= w.grad * 1e-5
  b-= b.grad * 1e-5

In [26]:
#Resetting the gradients
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

tensor([0., 0.])

In [28]:
#Training for 100 epochs using GD
for i in range(100):
  preds = model(inputs)
  loss = mse(preds,targets)
  loss.backward()
  with torch.no_grad():
      w -= w.grad * 1e-5
      b -= b.grad * 1e-5
      w.grad.zero_()
      b.grad.zero_()

In [29]:
preds = model(inputs)
loss = mse(preds,targets)
print(loss)

tensor(369.2569, grad_fn=<DivBackward0>)


##LR using Pytorch Built-ins

In [30]:
import torch.nn as nn

In [47]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [48]:
#TensorDataset for accessing rows from inputs and targets as tuples
from torch.utils.data import TensorDataset
train_ds = TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [49]:
#DataLoader fod splitting large data into many batches randomly
from torch.utils.data import DataLoader
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle=True)
for xb,yb in train_dl:
  print(xb)
  print(yb)

tensor([[ 92.,  87.,  64.],
        [ 68.,  96.,  71.],
        [103.,  43.,  36.],
        [ 73.,  67.,  43.],
        [ 73.,  66.,  44.]])
tensor([[ 82., 100.],
        [104., 118.],
        [ 20.,  38.],
        [ 56.,  70.],
        [ 57.,  69.]])
tensor([[ 88., 134.,  59.],
        [ 68.,  97.,  70.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 87., 135.,  57.]])
tensor([[118., 132.],
        [102., 120.],
        [ 81., 101.],
        [119., 133.],
        [118., 134.]])
tensor([[102.,  43.,  37.],
        [ 91.,  87.,  65.],
        [ 74.,  66.,  43.],
        [ 69.,  96.,  70.],
        [101.,  44.,  37.]])
tensor([[ 22.,  37.],
        [ 80., 102.],
        [ 57.,  69.],
        [103., 119.],
        [ 21.,  38.]])


In [50]:
#Define the model
model = nn.Linear(3,2) #3 inputs and 1 output
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3919, -0.1840,  0.3256],
        [ 0.2377, -0.0025,  0.5445]], requires_grad=True)
Parameter containing:
tensor([ 0.4384, -0.0818], requires_grad=True)


In [51]:
#Generate the predictions
preds = model(inputs)
preds

tensor([[30.7142, 40.5147],
        [40.7400, 56.1752],
        [28.7538, 51.8424],
        [44.5418, 44.1999],
        [32.5999, 54.1938],
        [31.2901, 40.7549],
        [41.2495, 56.7223],
        [29.4713, 52.6245],
        [43.9659, 43.9597],
        [32.5336, 54.5006],
        [31.2237, 41.0618],
        [41.3159, 56.4154],
        [28.2443, 51.2953],
        [44.6081, 43.8930],
        [32.0240, 53.9536]], grad_fn=<AddmmBackward0>)

In [52]:
#Inbuilt loss function
import torch.nn.functional as F #Contains all types of loss functions
loss_fn = F.mse_loss
loss = loss_fn(model(inputs),targets)
print(loss)

tensor(2941.0540, grad_fn=<MseLossBackward0>)


In [53]:
#Optimizer SGD -> Stochastic Gradient Descent
opt = torch.optim.SGD(model.parameters(), lr=1e-5)


In [54]:
#Utility function to train any model
def fit(num_epochs, model, loss_fn, opt, train_dl):
  for epoch in range(num_epochs):
    for xb,yb in train_dl:
      pred = model(xb)
      loss = loss_fn(pred,yb)
      loss.backward()#Computing grads
      opt.step()#Gradient Descent
      opt.zero_grad()#Resetting gradient
    if (epoch+1)%10 == 0 :
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


In [55]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100], Loss: 888.8426
Epoch [20/100], Loss: 392.8892
Epoch [30/100], Loss: 490.7452
Epoch [40/100], Loss: 89.2902
Epoch [50/100], Loss: 104.2296
Epoch [60/100], Loss: 61.0800
Epoch [70/100], Loss: 66.6533
Epoch [80/100], Loss: 12.8006
Epoch [90/100], Loss: 44.8461
Epoch [100/100], Loss: 21.5568


In [56]:
#Predicting for new inputs
model(torch.tensor([[75, 63, 44.]]))

tensor([[55.6715, 69.0191]], grad_fn=<AddmmBackward0>)